# 3 phase project

1. search reports; projected growth in interest and popluarity, and hopefully include facebook ad reach. useful for niche and product selection, also find the sentiment and polarity from twitter. For a real client I would customize the source for sentiment analysis.
2. product scoring; evaluate the quality of a given product on Aliexpress or Oberlo, will take into account the sentiment of reviews, as well as other metrics I will identify later.
3. Influencer scoring; predict overall sentiment and polarity of a given influencer as well as their trajectory in the next 6 to 12 months. Also calculate the similaritty of their audience with the client's audience. Lastly, include ig engagement stats too.

# How to carry out each phase
1. identify what data is needed for the functionality to be possible and collect it
2. identify what else may be good for a dropshipper to know and collect it
3. thorough EDA 
4. Build Models
5. Optimize
6. Deploy

# What data is nessecary?

1. google search trends via pytrends
2. number of people actually interested on google or facebook
3. predicted reach from a fb ad campaign
4. tweets regarding the topic via tweepy

Columns list: term, current popularity, covid percentage change, current [insert quantitative attr], predicted percentage change in the next [3, 6, 9, 12, 24] months, estimated reach, estimated clicks, sentiment, polarity

In [1]:
#imports and instantiations
from pytrends.request import TrendReq
import tweepy
from statsmodels.tsa import ar_model
import numpy as np
from textblob import TextBlob
import pandas as pd

In [10]:
#helper functions
def GetReport(keywords, span='today 5-y', geo='', quiet=True):
    pytrends = TrendReq(hl='en-US', tz=360)
    pytrends.build_payload(keywords, cat=0, timeframe=span, geo=geo, gprop='')
    ts = pytrends.interest_over_time().drop(['isPartial'], axis='columns')
    if quiet == False:
        print(ts.plot())
    return ts

def AnalyzeTwitter(keyword):
    c1 = 'aHXduTrDkva3ItY52tUtYVPvA'
    c2 = 'Qs6d4oNT3zXxDqOhita7IG07CfAJGceoqIs1sGuA4OURlbLP6d'
    a1 = '1181578611171762177-sGQaj7E9fpWi2aEB3MfWL4nTRovXYk'
    a2 = 'wa77yBJZJSOKOAzdaJYDruc9U1HrGhzyDhWgKvSQpm2hv'
    auth = tweepy.OAuthHandler(c1, c2)
    auth.set_access_token(a1, a2)
    api = tweepy.API(auth)
    topic = api.search(keyword)
    sent = 0
    sub = 0
    for i in range(len(topic)):
        tweet = topic[i]._json['text'].replace('@', '')
        blob = TextBlob(tweet)
        sent += blob.sentiment[0]/len(topic)
        sub += blob.sentiment[1]/len(topic)
    return sent, sub

def Collect(keyword):
    row = {}
    ts = GetReport([keyword])
    row['term'] = keyword
    model = ar_model.AutoReg(ts, lags=4).fit()
    pred = model.predict(start=260, end=356)
    current_popularity = np.array(ts)[-1][0]
    row['current_popularity'] =  current_popularity
    row['change_3mo'] = '{}%'.format(round(((pred[11] - current_popularity) / current_popularity) * 100, 1))
    row['change_6mo'] = '{}%'.format(round(((pred[23] - current_popularity) / current_popularity) * 100, 1))
    row['change_9mo'] = '{}%'.format(round(((pred[35] - current_popularity) / current_popularity) * 100, 1))
    row['change_12mo'] = '{}%'.format(round(((pred[47] - current_popularity) / current_popularity) * 100, 1))
    row['change_24mo'] = '{}%'.format(round(((pred[95] - current_popularity) / current_popularity) * 100, 1))
    estimated_reach = None
    estimated_clicks = None
    sentiment, subjectivity = AnalyzeTwitter(keyword)
    row['sentiment'] = round(sentiment, 2)
    row['subjectivity'] = round(subjectivity, 2)
    return row

def CollectLoop(terms_list):
    df = pd.DataFrame(Collect(terms_list[0]), index=[0])
    for term in terms_list[1:]:
        temp = pd.DataFrame(Collect(term), index=[0])
        df = pd.concat([df, temp])
    return df.reset_index().drop(['index'], axis='columns')

## Data Sourcing
in the bag: search trends, time series modeling, twitter nlp

need to figure out: facebook analytics, google analytics, fb ads?

Columns list: term, current popularity, covid percentage change, current [insert quantitative attr], predicted percentage change in the next [3, 6, 9, 12, 24] months, estimated reach, estimated clicks, sentiment, polarity

In [17]:
hobbies = ['biking', 'gardening', 'surfing', 'home design', 'workouts']
products = ['face mask', 'bike helmet', 'board shorts', 'back brace', 'puzzles']